# 0 - Instalation of modules

In [ ]:
#Run this cell to have the progress bar
!pip install tqdm
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

# 1 - Importing modules

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# 2 - Get all files needed

In [4]:
data_folder = "../../Dataset/Movie/"

rating = pd.read_csv(f"{data_folder}ratings.csv").drop(["timestamp"], axis=1)
movie = pd.read_csv(f"{data_folder}movies.csv")

data = pd.merge(movie, rating, on="movieId")

# 3 - Transforming data

In [6]:
#Séparation en ligne du dataframe en petits dataframe
N = 1000
data_list = []
L = data.shape[0]

for i in range(1,N+1):
    debut = int((i-1)*L/N)
    fin = int(i*L/N)
    data_list.append(data.iloc[debut:fin, :])

In [11]:
#pivot table de chaque petits dataframe
pv_list = []
N = len(data_list)

i = 0
for d in tqdm(data_list):
    pv_list.append(d.pivot_table(index="userId", columns="title", values="rating", dropna=False))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
#Réunir tous les pivots tables selon le film
titles = data.title.unique()
dico = {title : [] for title in titles}

for pv in pv_list:
    for title in pv:
        dico[title].append(pv[title])

In [24]:
#Concatener tous les pivots tables selon leur film et on les gardes uniquement si ils ont pas trop d'éléments manquant.
all_df = []
keep_title = []
thresh = 0.3

i = 0
for title in tqdm(titles):
    tmp = pd.concat(dico[title], axis=0)
    if not (np.sum(tmp.isna(), axis=0) > tmp.shape[0]*thresh):
        all_df.append(tmp)
        keep_title.append(title)


  0%|          | 0/10000 [00:00<?, ?it/s]

In [57]:
final_df = pd.DataFrame(index=np.sort(data.userId.unique()), columns=keep_title)

In [62]:
final_df

,"Matrix, The (1999)",Forrest Gump (1994),Pulp Fiction (1994),Star Wars: Episode IV - A New Hope (1977),Back to the Future (1985),Jurassic Park (1993),"Silence of the Lambs, The (1991)","Shawshank Redemption, The (1994)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Star Wars: Episode VI - Return of the Jedi (1983),...,Harry Potter and the Half-Blood Prince (2009),"King's Speech, The (2010)",Gravity (2013),Star Wars: Episode VII - The Force Awakens (2015),How to Train Your Dragon (2010),"Piano, The (1993)",Sense and Sensibility (1995),Arrival (2016),Whiplash (2014),Intouchables (2011)
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
all_df[0].name

'Matrix, The (1999)'

In [36]:
for df in tqdm(all_df):
    index = df.index
    values = df.values
    title = all_df[0].name

    for id, val in tqdm(zip(index, values)):
        final_df[id-1] = values

Matrix, The (1999)                           NaN
Forrest Gump (1994)                          NaN
Pulp Fiction (1994)                          NaN
Star Wars: Episode IV - A New Hope (1977)    NaN
Back to the Future (1985)                    NaN
                                            ... 
Piano, The (1993)                            NaN
Sense and Sensibility (1995)                 NaN
Arrival (2016)                               NaN
Whiplash (2014)                              NaN
Intouchables (2011)                          NaN
Name: 1, Length: 131, dtype: object

In [12]:
cdata = data.copy()
N = 10**4
thresh = int(N*0.3)

ids = cdata["userId"].value_counts()[:N].index
cdata = cdata[cdata["userId"].isin(ids)]

titles = cdata["title"].value_counts()[:N].index
cdata = cdata[cdata["title"].isin(titles)]

pv = cdata.pivot_table(index="userId", columns="title", values="rating")
pv = pv.dropna(axis=1, thresh=thresh)

pv

title,10 Things I Hate About You (1999),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",50 First Dates (2004),8 Mile (2002),A.I. Artificial Intelligence (2001),...,X-Men (2000),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Zodiac (2007),Zombieland (2009),Zoolander (2001),xXx (2002)
userId,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,5.0,3.0,4.0,3.0,3.0,3.5,NaN,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,4.0,3.5,3.5
12,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
72,5.0,3.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN
80,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,5.0,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162484,3.0,3.0,NaN,4.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,3.5,NaN
162495,2.0,1.5,NaN,1.0,0.5,3.0,3.5,3.0,NaN,2.5,...,4.0,NaN,2.5,4.0,NaN,NaN,NaN,5.0,3.5,NaN
162508,3.0,NaN,NaN,NaN,5.0,NaN,4.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN


In [14]:
np.sum(pv.isna(), axis=0)*100/pv.shape[0]

title
10 Things I Hate About You (1999)                         61.29
101 Dalmatians (One Hundred and One Dalmatians) (1961)    67.12
12 Angry Men (1957)                                       57.98
2001: A Space Odyssey (1968)                              33.39
28 Days Later (2002)                                      49.24
                                                          ...  
Young Frankenstein (1974)                                 56.11
Zodiac (2007)                                             66.56
Zombieland (2009)                                         63.09
Zoolander (2001)                                          54.12
xXx (2002)                                                69.79
Length: 718, dtype: float64

<p>Beaucoup de film sont très peu rempli, il faudrait faire un algorithme qui permet de faire un pivot table sur toutes les données.</p>
<p>Pour cela il faudrait découper le df par les colonnes, du style pleins de df en (:, 10) un truc comme ça.</p>
<p>puis on applique le dropna et à la fin on merged tous les df.</p>

In [ ]:
def add_only_new_elem(base_list:list, adding_list:list)->list:
    commun = [elem for elem in adding_list if elem in base_list]
    for elem in commun:
        adding_list.remove(elem)
    return adding_list

In [ ]:
genre = []
for g in data.genres.value_counts().index:
    genre = genre + add_only_new_elem(genre, g.split("|"))
genre.remove("IMAX")
genre.remove('(no genres listed)')